
[Dataset](https://drive.google.com/file/d/1wWsrN2Ep7x6lWqOXfr4rpKGYrJhWc8z7/view)

In [59]:
import os
Root = "/content/drive/MyDrive/SPEECH Data set"
os.chdir(Root)

In [60]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [81]:
ls

Actor_01/  Actor_05/  Actor_09/  Actor_13/  Actor_17/  Actor_21/
Actor_02/  Actor_06/  Actor_10/  Actor_14/  Actor_18/  modelForPrediction1.sav
Actor_03/  Actor_07/  Actor_11/  Actor_15/  Actor_19/
Actor_04/  Actor_08/  Actor_12/  Actor_16/  Actor_20/


In [62]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [63]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [64]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [65]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/SPEECH Data set/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [66]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [67]:
x_train

array([[-5.94312073e+02,  2.16505127e+01, -9.47290039e+00, ...,
         2.34925552e-04,  1.95549364e-04,  9.81910125e-05],
       [-4.83997894e+02,  2.02410831e+01, -1.78483315e+01, ...,
         3.16655525e-04,  2.72986013e-04,  1.67585837e-04],
       [-5.04864716e+02,  3.52971039e+01, -1.44038277e+01, ...,
         6.08151604e-04,  5.55269769e-04,  4.47782280e-04],
       ...,
       [-7.95570374e+02,  3.55506325e+01,  1.64709556e+00, ...,
         6.26254950e-06,  4.75230536e-06,  2.51864594e-06],
       [-3.78953918e+02,  2.54936085e+01, -2.97894955e+01, ...,
         1.41517830e-03,  8.01826653e-04,  5.08085592e-04],
       [-6.06880249e+02,  5.45934029e+01,  1.42014236e+01, ...,
         1.52038425e-04,  9.58592282e-05,  3.05187023e-05]])

In [68]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(537, 180)


In [69]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [70]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [71]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [72]:
#Predict for the test set
y_pred=model.predict(x_test)

In [73]:
y_pred

array(['disgust', 'calm', 'disgust', 'happy', 'calm', 'calm', 'fearful',
       'fearful', 'happy', 'fearful', 'happy', 'calm', 'disgust', 'happy',
       'disgust', 'disgust', 'disgust', 'disgust', 'fearful', 'calm',
       'fearful', 'calm', 'happy', 'happy', 'happy', 'happy', 'calm',
       'disgust', 'calm', 'fearful', 'disgust', 'calm', 'calm', 'calm',
       'calm', 'fearful', 'fearful', 'disgust', 'calm', 'fearful', 'calm',
       'disgust', 'calm', 'disgust', 'calm', 'disgust', 'happy',
       'fearful', 'happy', 'fearful', 'calm', 'fearful', 'calm', 'calm',
       'disgust', 'calm', 'disgust', 'calm', 'calm', 'happy', 'disgust',
       'disgust', 'disgust', 'disgust', 'disgust', 'disgust', 'calm',
       'disgust', 'calm', 'calm', 'calm', 'calm', 'disgust', 'fearful',
       'disgust', 'calm', 'disgust', 'disgust', 'disgust', 'fearful',
       'calm', 'happy', 'disgust', 'disgust', 'fearful', 'fearful',
       'fearful', 'disgust', 'fearful', 'happy', 'fearful', 'calm',
      

In [87]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 67.22%


In [75]:
from sklearn.metrics import accuracy_score, f1_score

In [76]:
f1_score(y_test, y_pred,average=None)

array([0.72072072, 0.62135922, 0.68421053, 0.65714286])

In [83]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(45)

,Actual,Predicted
0,disgust,disgust
1,calm,calm
2,fearful,disgust
3,happy,happy
4,calm,calm
5,happy,calm
6,fearful,fearful
7,fearful,fearful
8,happy,happy
9,calm,fearful


In [78]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [96]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("//content/drive/MyDrive/SPEECH Data set/Actor_01/03-01-01-01-01-01-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['calm'], dtype='<U7')

In [80]:
feature

array([[-6.93497009e+02,  5.00643921e+01,  5.71450531e-01,
         1.43299656e+01,  3.33637071e+00, -2.54071975e+00,
        -4.05790949e+00, -1.07119999e+01, -7.29413939e+00,
         1.74018872e+00, -4.19064236e+00,  1.95466173e+00,
        -5.24789381e+00,  2.78142977e+00, -3.16756773e+00,
        -3.40008307e+00, -2.37803197e+00, -5.68717539e-01,
        -6.47753334e+00, -1.24320555e+00, -2.80542517e+00,
        -5.43635798e+00, -4.46875393e-01, -3.63516593e+00,
        -2.98372602e+00, -5.63902557e-01, -1.65101993e+00,
        -5.55944741e-01, -3.41018438e+00, -2.24465466e+00,
        -3.13058877e+00, -2.70089960e+00, -1.88821292e+00,
        -5.54154456e-01, -3.96459913e+00, -2.13485193e+00,
        -3.94577074e+00, -1.62457860e+00, -2.03990698e+00,
        -3.62910867e+00,  6.33751631e-01,  6.48747087e-01,
         6.21731520e-01,  6.34555459e-01,  6.60326064e-01,
         6.60485923e-01,  7.00930357e-01,  7.31556237e-01,
         7.46886790e-01,  7.23434567e-01,  7.16107428e-0